In [6]:
# Import forecasts with specific start dates and calculate production and decline rates at a date in the future

import numpy as np
import pandas as pd

def arps_decline(UID, phase, Qi, Dei, Def, b, t, prior_cum, prior_t):
    # UID is a unique identifier for the well such as API, must be a number
    # phase is 1 = oil, 2 = gas, or 3 = water
    # Qi is the initial production rate typically in bbl/day or Mcf/day
    # Dei is the initial effective annual decline rate
    # Def is the final effective annual decline rate at which point the decline becomes exponential
    # b is the b-factor used in hyperbolic or harmonic decline equations
    # t is the time as a month integer
    # prior_cum is the cumulative amount produced before the start of the decline calcuations
    # prior_t is an integer representing the final month from a previous decline segment
    
    # Calculations to determine decline type
    if Dei == Def:
        Type = 'exp'
    elif Dei > Def and b == 1:
        Type = 'har'
        Dn = Dei / (1 - Dei)
        Qlim = Qi * ((-np.log(1 - Def)) / Dn)
        tlim = (((Qi / Qlim) - 1) / Dn) * 12 # output in months
    else:
        Type = 'hyp'
        Dn = (1 / b) * (((1 - Dei) ** -b) - 1)
        Qlim = Qi * ((-np.log(1 - Def)) / Dn) ** (1 / b)
        tlim = ((((Qi / Qlim) ** b) - 1) / ( b * Dn)) * 12 # output in months
    
    # Generate volumes
    if Type == 'hyp':
        Dn_t = Dn / (1 + b * Dn * (t / 12))
        De_t = 1 - (1 / ((Dn_t * b) + 1)) ** (1 / b)
        if De_t > Def:
            q = Qi * (1 + b * Dn * (t / 12)) ** (-1/b)
            Np = ((Qi ** b) / (Dn * (1 - b))) * ((Qi ** (1 - b)) - (q ** (1 - b))) * 365
        else:
            q = Qlim * np.exp(-(-np.log(1 - Def)) * ((t - tlim) / 12))
            Np = ((Qlim - q) / (-np.log(1 - Def)) * 365) + (((Qi ** b) / 
                    (Dn * (1 - b))) * ((Qi ** (1 - b)) - (Qlim ** (1 - b))) * 365)
            De_t = Def
    elif Type == 'har':
        Dn_t = Dn / (1 + Dn * (t / 12))
        De_t = 1 - (1 / (Dn_t + 1))
        if De_t > Def:
            q = Qi / (1 + b * Dn * (t / 12))
            Np = (Qi / Dn) * np.log(Qi / q) * 365
        else:
            q = Qlim * np.exp(-(-np.log(1 - Def)) * ((t - tlim) / 12))
            Np = ((Qlim - q) / (-np.log(1 - Def)) * 365) + ((Qi / Dn) * np.log(Qi / Qlim) * 365)
            De_t = Def
    else:
        q = Qi * np.exp(-(-np.log(1 - Dei)) * (t / 12))
        Np = (Qi - q) / (-np.log(1 - Dei)) * 365
        De_t = Dei
    
    return UID, phase, t + prior_t, q, De_t

In [17]:
# Import data for decline parameters
fcst_list = pd.read_csv('SCST_Forecast.csv')
fcst_list['Start_Date'] = fcst_list['Start_Date'].astype('datetime64[ns]') #dates must be formatted as yyyy-mm-dd
fcst_list['Phase'] = np.where(fcst_list['Phase'] == 'oil', 1, 2)
fcst_list = fcst_list.query('IP3 > 1 & Dei >= Def & b > 0 & Scenario == "SCST_CC_072021"')
C = fcst_list.keys() # Columns
R = fcst_list.index # Rows

In [25]:
# Loop through DataFrame and output monthly volumes and oneline

RF_Date = pd.to_datetime('2021-07')
out_nparr = np.empty([5, 0])

for r in R:
    RF_Months = int(round((RF_Date - fcst_list.loc[r, 'Start_Date']) / np.timedelta64(1, 'M'), 0))
    result = arps_decline(fcst_list.loc[r, 'Property_ID'], fcst_list.loc[r, 'Phase'], fcst_list.loc[r, 'IP3'], 
                          fcst_list.loc[r, 'Dei'], fcst_list.loc[r, 'Def'], fcst_list.loc[r, 'b'], RF_Months, 0, 0)
    out_nparr = np.column_stack((out_nparr, result))
    

# Create pandas dataframe from array and output results to csv

out_RF = pd.DataFrame(np.transpose(out_nparr), columns = ['UID', 'phase', 'Months', 'Rate', 'Dei'])
out_RF['RF_Date'] = RF_Date
out_RF = pd.merge(out_RF, fcst_list[['Property_ID', 'b', 'Def']], how = 'inner', left_index = True, 
                  right_index = True)
out_RF.to_csv('dca_roll_forward.csv')
out_RF.head()

,UID,phase,Months,Rate,Dei,RF_Date,Property_ID,b,Def
0,3.501124e+09,1.0,23.0,80.479070,0.315982,2021-07-01,3501124067,0.950259,0.08
1,3.501124e+09,2.0,23.0,522.075072,0.265540,2021-07-01,3501124067,1.035511,0.06
2,3.507325e+09,1.0,16.0,24.325660,0.270283,2021-07-01,3507325178,0.907839,0.08
3,3.507325e+09,2.0,27.0,224.569049,0.188856,2021-07-01,3507325178,1.053566,0.06
4,3.501724e+09,2.0,19.0,152.982461,0.172603,2021-07-01,3501724093,1.050975,0.06
